In [ ]:
def bub_sort(arr):
    n = len(arr)
    for i in range(n):
            swapped = False
            for j in range(0, n-i-1):
                if arr[j] > arr[j+1]:
                    arr[j], arr[j+1] = arr[j+1], arr[j]
                    swapped = True
            if not swapped:
                break
    return arr

        

Sorted array is: [11, 12, 22, 25, 34, 64, 90]


In [8]:
def select_sort(arr):
    n = len(arr)
    for i in range(n):
        min_idx = i
        for j in range(i+1, n):
            if arr[j] < arr[min_idx]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
    return arr

In [ ]:
def inser_sort(arr):
    

In [9]:
if __name__ == "__main__":
    arr = [64, 34, 25, 12, 22, 11, 90]
    sorted_arr = bub_sort(arr)
    print("Sorted array is:", sorted_arr)
    select_sort(arr)
    print("Sorted array is:", arr)
        

Sorted array is: [11, 12, 22, 25, 34, 64, 90]
Sorted array is: [11, 12, 22, 25, 34, 64, 90]
